https://github.com/ilkayDevran/Gaussian_Filtering/tree/456b41df03761546cd61634caaeb7817478dfd47

In [1]:
#import other
%run other.ipynb
%run muti_filter.ipynb

In [2]:
import numpy as np
import time
import math
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

In [3]:
def Main():
    # 設定參數
    large_mesh = 11
    n = 2   #kernel
    # 匯入圖片
    image_path = 'source/gaussian-noise.jpg'
    init_image = other_class.input_image(image_path)
    # 把圖片切成大 mesh
    input_image = other_class.to_mesh(init_image, large_mesh)
    #--------------------------------------------------------------------------
    
    # 高斯濾波 Variance=2 kernel=2*2
    input_image_gaussian = gaussian_filter_class.gaussian_filter(input_image, init_image, 2, n=2)
    
    #--------------------------------------------------------------------------
    # mesh轉成array
    output_image = other_class.mesh_to_array(input_image_gaussian, init_image, large_mesh)
    # 並將array轉成圖片，並儲存
    show_image = Image.fromarray(output_image, 'RGB')
    show_image.save('result/new_cat.png')

In [4]:
if __name__ == '__main__':
    Main()

x_scale: 22
y_scale: 19
